In [1]:
import numpy as np
import pandas as pd

In [14]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

In [9]:
df=pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day32-binning-and-binarization/train.csv',usecols=['Age','Fare','Survived'])

In [10]:
df.dropna(inplace=True)

In [11]:
x=df.iloc[:,1:]
y=df.iloc[:,-0]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
X_train.head(2)

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542


In [15]:
clf=DecisionTreeClassifier()

In [16]:
clf.fit(X_train, y_train)
Y_pred = clf.predict(X_test)

In [17]:
accuracy_score(y_test, Y_pred)

0.6153846153846154

In [18]:
np.mean(cross_val_score(DecisionTreeClassifier(), x, y, cv=10, scoring='accuracy'))

np.float64(0.6317683881064162)

In [19]:
kbin_age= KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='quantile')
kbin_fare= KBinsDiscretizer(n_bins=15, encode='ordinal', strategy='quantile')

In [20]:
trf=ColumnTransformer(transformers=[
    ('first', kbin_age, [0]),
    ('second', kbin_fare, [1])
])


In [21]:
X_train_trf=trf.fit_transform(X_train)
X_test_trf=trf.transform(X_test)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


In [22]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [23]:
output=pd.DataFrame({'age':X_train['Age'], 'fare':X_train['Fare'], 'age_binned':X_train_trf[:,0], 'fare_binned':X_train_trf[:,1]})

In [24]:
output['age_binned']=pd.cut(x=X_train['Age'], bins=trf.named_transformers_['first'].bin_edges_[0])
output['fare_binned']=pd.cut(x=X_train['Fare'], bins=trf.named_transformers_['second'].bin_edges_[0])

In [25]:
output.sample(5)

,age,fare,age_binned,fare_binned
286,30.0,9.5000,"(28.0, 30.0]","(8.158, 10.5]"
116,70.5,7.7500,"(54.0, 80.0]","(7.25, 7.775]"
130,33.0,7.8958,"(32.0, 35.0]","(7.775, 7.896]"
757,18.0,11.5000,"(16.0, 19.0]","(10.5, 13.0]"
125,12.0,11.2417,"(6.0, 16.0]","(10.5, 13.0]"


In [26]:
clf=DecisionTreeClassifier()
clf.fit(X_train_trf, y_train)
Y_pred2 = clf.predict(X_test_trf)

In [27]:
accuracy_score(y_test, Y_pred)

0.6153846153846154

In [28]:
X_trf=trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(), x, y, cv=10, scoring='accuracy'))

/home/codespace/.local/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


np.float64(0.6401017214397495)